# Jina Search

This notebook goes over how to use the Jina Search tool.
Go to the [Jina Website](https://jina.ai) for more information about Jina AI and their offerings.

In [ ]:
%pip install --upgrade --quiet langchain-community

In [1]:
from langchain_community.tools import JinaSearch

In [2]:
search = JinaSearch()

In [3]:
search.run("what is langchain?")

'[{"title": "LangChain", "link": "https://www.langchain.com/", "snippet": "<strong>LangChain</strong>\\u2019s suite of products supports developers along each step of their development journey.", "content": "From startups to global enterprises,  \\nambitious builders choose  \\nLangChain products.\\n--------------------------------------------------------------------------------------\\n\\n![Image 1](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/65ca3b7c22746faa78338532_logo_Ally.svg)![Image 2](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/65ca3b7c08e67bb7eefba4c2_logo_Rakuten.svg)![Image 3](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/65ca3b7c576fdde32d03c1a0_logo_Elastic.svg)![Image 4](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/65ca3b7c6d5592036dae24e5_logo_BCG.svg)![Image 5](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/6651232f52288a15729f1654_streamlit-logo-B405F7E2FC-seeklogo.com.png)![Image 6](https://cdn.